In [1]:
!pip install -r requirements.txt

In [2]:
## Import libraries and enable GPU in tensorflow

import tensorflow as tf
try:
    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

ImportError: cannot import name 'LayerNormalization' from 'tensorflow.python.keras.layers.normalization' (C:\Users\User\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\layers\normalization\__init__.py)

In [ ]:
# Import libraries

import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

# Compile and Train
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# Define the dataset path

data_dir = "./dogImages/train"
print(data_dir)

In [ ]:
# Degine some parameter
img_height = 224
img_width = 224
batch_size = 128 ## Images will pass into the network per epochs

# Load images into the train dataloader
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  r"C:\Users\User\Documents\GitHub\dog-project\dogImages\dogImages\train",
  labels='inferred',
  label_mode='categorical',
  # validation_split=0.2, #use 80% as training dataset
  # subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

class_names = train_ds.class_names

In [ ]:
# Degine some parameter
img_height = 224
img_width = 224
batch_size = 1 ## Images will pass into the network per epochs

# Load images into the validation dataloader
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  r"C:\Users\User\Documents\GitHub\dog-project\dogImages\dogImages\valid",
  labels='inferred',
  label_mode='categorical',
  # validation_split=0.2, #use 20% as training dataset
  # subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

class_names = val_ds.class_names

In [ ]:
# Show images to make sure images loaded are correct

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
# Check the dataloader parameter

for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
# Normalize the images
## normal images => 0-255
## normalized images => 0-1

## example:
## 127 in normal image === 0.5 in normalized image

## we normalize images to reduce the computation cost.

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# create the base model
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False)
base_model

In [ ]:
# freeze layers -> we train again from scratch
for layer in base_model.layers[:143]:
    layer.trainable = False

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
       print(i, layer.name, "-", layer.trainable)

In [ ]:
import tensorflow.keras as K

# this is the model we will train
model = K.models.Sequential()
model.add(K.layers.Lambda(lambda image: tf.image.resize(image, (224, 224))))
model.add(base_model)
model.add(K.layers.Flatten())
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(256, activation='relu'))
model.add(K.layers.Dropout(0.5))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(128, activation='relu'))
model.add(K.layers.Dropout(0.5))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(64, activation='relu'))
model.add(K.layers.Dropout(0.5))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(133, activation='softmax')) # 3 is number of output

In [ ]:
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy']) # Define optimizer/LR

In [ ]:
EPOCHS = 100 # number of cycle

checkpoint = ModelCheckpoint('dog_vgg.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

callbacks = [checkpoint]

In [ ]:
# start training
history = model.fit(train_ds, validation_data=val_ds,
          epochs=EPOCHS, callbacks=callbacks)

In [ ]:
# visualize the result (acc, val_acc, loss, val_loss)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('dog_vgg.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("dog_vgg.tflite", "wb").write(tflite_model)

In [ ]:
interpreter = tf.lite.Interpreter(model_path = "dog_vgg.tflite")
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])